In [1]:
# 2020-05-13
# Redoing all gene list creation, stringent and non-stringent
# Want two lists, one ENS and one HGNC
# convert each individual data stream from whatever source it is in to the other, flow that into the joint lists


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import mygene
import pickle

In [3]:
p_d = '/Users/cweiss19/Box Sync/MPRA/DATA/loci_to_genes/2020_05_13_bothlists'
our_vars_file = '/Users/cweiss19/Box Sync/MPRA/VARs_anno.xlsx'

# GeneHancer
GH_connections = f'{p_d}/GeneHancer_gene_associations_scores.txt'
GH_enhancers = f'{p_d}/GeneHancer_elements.txt'

# HiC
HiC = f'{p_d}/Jung_2019_Supp_table_3.xlsx'

# eQTL and overlapping promoters in VARs_anno.xlsx

# HGNC <-> ENSid
ens_to_hgnc = f'{p_d}/ens--hgnc_conv_dict.pickle'
hgnc_to_ens = f'{p_d}/hgnc--ens_conv_dict.pickle'


In [4]:
# load conversion dicts
with open(ens_to_hgnc, 'rb') as handle:
    e_to_h_d = pickle.load(handle)  # {ensID: 'hgnc1,hgnc2', ensID2: 'hgnc3'} etc

with open(hgnc_to_ens, 'rb') as handle:
    h_to_e_d = pickle.load(handle)   # same as above but hgnc keys
    
    
# all the ensIDs that are in one are in the other, same for hgnc

In [5]:
# define ultimate function to convert genes from one to the other, applied to pandas DF rows

def convert(r, which_key, column):
    if which_key == 'ens':
        d = e_to_h_d
    if which_key == 'hgnc':
        d = h_to_e_d
    #print(r)
    genes = r[column]  # will be single gene, or genes separated by ','
    if not genes == 0:
        genes_l = genes.split(',')
        converted_genes_l = []
        for g in genes_l:
            if g in d:
                conv_g = d[g]  # could be single gene, or genes separated by ','
                conv_g_l = conv_g.split(',')  # turn into list
                converted_genes_l.extend(conv_g_l)
        converted_genes = ','.join(list(set(converted_genes_l)))
        if len(converted_genes) > 0:
            return converted_genes
        else: 
            return 0
    else:
        return 0

In [6]:
df_vars = pd.read_excel(
    our_vars_file, 
    index_col=0, 
    header=2, 
    sheet_name=0
    )
df_vars.drop(df_vars.index[0], inplace=True)

In [7]:
# HiC overlap

dist_from_var_HiC = 10000

def read_in_excel(filepath):
    df = pd.read_excel(filepath,
        index_col=None,
        header=0,
        sheet_name=0)
    return df

def rows_to_dict(df, dict1):
    for index, row in df.iterrows():
        pos = row['Interacting_fragment']
        gene = row['Promoter']
        if pos in dict1:
            dict1[pos].append(str(gene))
        else:
            dict1[pos] = [str(gene)]
    return dict1

def create_range_dict(dict1, dict2):
    keys = list(dict1.keys())
    for each_key in keys:
        split_pos = each_key.split('.')
        chromosome = split_pos[0]
        if chromosome in dict2:
            dict2[chromosome].append(each_key)
        else:
            dict2[chromosome] = [each_key]
    return dict2

def find_overlapping_ranges(position, position_dict, chromosome, how_close):
    ranges_to_search = position_dict[chromosome]  # only look within the same chromosome
    overlapping_ranges = []  # to store the ranges that overlap the query position
    ranges_to_search_sorted = sorted(ranges_to_search)
    x1 = position - how_close
    x2 = position + how_close
    for each_range in ranges_to_search_sorted:
        range_split = each_range.split('.')
        y1 = int(range_split[1])
        y2 = int(range_split[2])
        if x1 <= y2 and y1 <= x2:
            overlapping_ranges.append(each_range)
        if y1 > x2:  # beginning of search range is beyond end of query range, no point in searching past this
            break
    return overlapping_ranges

def connect_back_to_genes(dict1, dict2, dict3):
    for each_chr, positions in dict1.items():
        for each_pos, data in positions.items():
            genes = []
            for each_range in data:
                connected_gene = dict2[each_range]
                genes.append(connected_gene)
            new_key = f'{each_chr}_{each_pos}'
            flat_genes = [item for sublist in genes for item in sublist]
            flat_genes = list(set(flat_genes))   # added to output only unique gene lists
            gene_string = ",".join(flat_genes)
            dict3[new_key] = gene_string
    return dict3

def filter_df(df, col, filter_value):
    filtered = df.loc[df[col] == filter_value]
    return filtered

p_o = read_in_excel(HiC)  # read in gene:interactingfragment:tissue df
esc_p_o = filter_df(p_o, 'Tissue_type', 'Human ES cells')
npc_p_o = filter_df(p_o, 'Tissue_type', 'Neural Progenitor Cell')
msc_p_o = filter_df(p_o, 'Tissue_type', 'Mesenchymal Stem Cell')

allcols = []
for colname, d in {'genes': p_o, 'esc_genes': esc_p_o, 'npc_genes': npc_p_o, 'msc_genes': msc_p_o}.items():
    p_o_dict = rows_to_dict(d, {})  # dict where keys are chr.start.end and value is a list of promoters interacting with it
    p_o_pos_dict = create_range_dict(p_o_dict, {})    # dict of chr: chr.start.end of ranges

    chr_grouped = df_vars.groupby('chromosome')
    chr_output = {}
    for each_chromosome, variants in chr_grouped:
        positions = variants['pos_hg19'].tolist()
        chr_output[each_chromosome] = {x: find_overlapping_ranges(int(x), p_o_pos_dict, each_chromosome, dist_from_var_HiC) for x in positions}   # pos: [chr.start.end] from ranges

    to_genes = connect_back_to_genes(chr_output, p_o_dict, {})
    print(len(to_genes.values()))
    print(sum(len(x)!=0 for x in to_genes.values()))
    output_df = pd.DataFrame.from_dict(to_genes, orient='index')
    output_df.columns = [colname]
    allcols.append(output_df)

# join all output dfs
output_df_HiC = pd.concat(allcols, axis=1)
#print(output_df_HiC[output_df_HiC['genes'] != ''])


# flatten the lists of multiple genes for each variant

output_df_HiC['oligo'] = df_vars.index.tolist()
output_df_HiC = output_df_HiC.set_index('oligo')
output_df_HiC = output_df_HiC.replace('', 0)

def clean_hic_lists(r, column):
    genes = r[column]
    if not genes == 0:
        genes_l = genes.split(';')
        clean_gs = ','.join(genes_l)
        return clean_gs
    else: 
        return 0

output_df_HiC['genes'] = output_df_HiC.apply(lambda row: clean_hic_lists(row, 'genes'), axis=1)
output_df_HiC['esc_genes'] = output_df_HiC.apply(lambda row: clean_hic_lists(row, 'esc_genes'), axis=1)
output_df_HiC['msc_genes'] = output_df_HiC.apply(lambda row: clean_hic_lists(row, 'msc_genes'), axis=1)
output_df_HiC['npc_genes'] = output_df_HiC.apply(lambda row: clean_hic_lists(row, 'npc_genes'), axis=1)
#print(output_df_HiC[output_df_HiC['esc_genes'] != 0].tail(5))

# convert each column to ensID
output_df_HiC['genes_ens'] = output_df_HiC.apply(lambda row: convert(row, 'hgnc', 'genes'), axis=1)
output_df_HiC['esc_genes_ens'] = output_df_HiC.apply(lambda row: convert(row, 'hgnc', 'esc_genes'), axis=1)
output_df_HiC['msc_genes_ens'] = output_df_HiC.apply(lambda row: convert(row, 'hgnc', 'msc_genes'), axis=1)
output_df_HiC['npc_genes_ens'] = output_df_HiC.apply(lambda row: convert(row, 'hgnc', 'npc_genes'), axis=1)

# check
print(output_df_HiC[output_df_HiC['esc_genes_ens'] != 0].tail(2))


14042
4797
14042
1922
14042
639
14042
1851
                                                                          genes  \
oligo                                                                             
libB_seq108668_[chrX:152673474-152673673]                     PNMA3,ZFP92,PNMA5   
libA_seq46943_[chrX:153178234-153178433]   NAA10,RENBP,ARHGAP4,IDH3G,SRPK3,SSR4   

                                                  esc_genes npc_genes  \
oligo                                                                   
libB_seq108668_[chrX:152673474-152673673]             ZFP92     ZFP92   
libA_seq46943_[chrX:153178234-153178433]   IDH3G,SRPK3,SSR4         0   

                                          msc_genes  \
oligo                                                 
libB_seq108668_[chrX:152673474-152673673]     ZFP92   
libA_seq46943_[chrX:153178234-153178433]          0   

                                                                                   genes_ens  \
oligo         

In [8]:
print(len(output_df_HiC[output_df_HiC['genes_ens'] != 0]))

4688


In [9]:
## Genehancer data 

dist_fr_var_GH = 10000 # want to look for enhancer-gene connections in GeneHancer within 10kb of our variant (10kb on either side)

def find_overlapping_ranges(position, position_dict, chromosome, how_close):    # position dict: {'chr': 'start.end'}
    info = position_dict[chromosome]  # only look within the same chromosome
    overlapping_ranges = []  # to store the ranges that overlap the query position
    ranges_to_search = list(info)  # list of positions to search through
    ranges_to_search_sorted = sorted(ranges_to_search)
    x1 = position - how_close
    x2 = position + how_close
    for each_range in ranges_to_search_sorted:
        range_split = each_range.split('.')
        y1 = int(range_split[1])
        y2 = int(range_split[2])
        if x1 <= y2 and y1 <= x2:
            overlapping_ranges.append(info[each_range])  # append the GHID
        if y1 > x2:  # beginning of search range is beyond end of query range, no point in searching past this
            break
    return overlapping_ranges

e_df = pd.read_csv(GH_enhancers, sep='\t', header=0, index_col='GHid')  # enhancers, their locations, and GHids
e_df['GHid'] = e_df.index.tolist()
e_df['chr'] = 'chr' + e_df['chr'].astype(str)
e_df['chrom_start_end'] = e_df['chr'].astype(str) + '.' + e_df['element_start'].astype(str) + '.' + e_df['element_end'].astype(str)
#print(e_df.head())

vg_df = pd.read_csv(GH_connections, sep='\t', header=0, index_col=False)  # enhancer-gene connections, the gene and whether connection is elite or not
vg_df['is_enhancer_elite'] = vg_df['GHid'].map(e_df['is_elite'])   # add col to enhancer-gene connection table which says whether the enhancer itself is elite or not
vg_df['chr'] = vg_df['GHid'].map(e_df['chr'])
vg_df['e_start'] = vg_df['GHid'].map(e_df['element_start'])
vg_df['e_end'] = vg_df['GHid'].map(e_df['element_end'])
vg_df['chrom_start_end'] = vg_df['chr'].astype(str) + '.' + vg_df['e_start'].astype(str) + '.' + vg_df['e_end'].astype(str)

vg_df.set_index(['GHid', 'symbol'], inplace=True)
vg_df.sort_index(inplace=True)
vg_df.drop(columns=['combined_score'], inplace=True)

# to use with stringent list
filtered_vg_df = vg_df[(vg_df['is_elite'] == 1)]   # only filtering on v-g connections


pos_dict = {}   # chromosome: {chr.start.end: GHID}
chrom_grouped = e_df.groupby('chr')
for each_chr, grp in chrom_grouped:
    cs = dict(zip(grp['chrom_start_end'].tolist(), grp['GHid'].tolist()))   # {chrom.start.end: GHID}
    pos_dict[each_chr] = cs

chrom_grouped = df_vars.groupby('chromosome')  # find overlapping ranges in our variants by chromosome
chr_output = {}
for each_chr, variants in chrom_grouped:
    positions = variants['pos_hg38'].tolist()
    positions = [x for x in positions if not np.isnan(x)]
    chr_output[each_chr] = {x: find_overlapping_ranges(int(x), pos_dict, each_chr, dist_fr_var_GH) for x in positions if x != 'None'}   # pos: [GHID] from ranges

to_genes_ns = {}
to_genes_s = {}

for each_chromosome, positions in chr_output.items():
    for each_position, connections in positions.items():
        x_ns = []
        x_s = []
        for each_connection in connections:   # each_connection is the GHID
            if each_connection in filtered_vg_df.index:
                connected_genes_df_s = filtered_vg_df.loc[each_connection, :]
                connected_genes_s = connected_genes_df_s.index.tolist()  # get the symbols from the other layer of multiindex
                x_s.append(connected_genes_s)
            if each_connection in vg_df.index:
                connected_genes_df_ns = vg_df.loc[each_connection, :]
                connected_genes_ns = connected_genes_df_ns.index.tolist()
                x_ns.append(connected_genes_ns)
            
        a_ns = [gene for sublist in x_ns for gene in sublist if gene != '']
        connected_genes_clean_ns = list(set(a_ns))
        connected_genes_clean_string_ns = ','.join(connected_genes_clean_ns)
        to_genes_ns[f'{each_chromosome}_{each_position}'] = connected_genes_clean_string_ns
        
        a_s = [gene for sublist in x_s for gene in sublist if gene != '']
        connected_genes_clean_s = list(set(a_s))
        connected_genes_clean_string_s = ','.join(connected_genes_clean_s)
        to_genes_s[f'{each_chromosome}_{each_position}'] = connected_genes_clean_string_s
        
output_df_GH = pd.DataFrame(index=df_vars.index.tolist())        
output_df_GH['GH_genes_ns'] = df_vars['chr_pos38'].map(to_genes_ns)
output_df_GH['GH_genes_s'] = df_vars['chr_pos38'].map(to_genes_s)

output_df_GH['GH_genes_ns'].fillna(0, inplace=True)
output_df_GH['GH_genes_ns'].replace('',0, inplace=True)
output_df_GH['GH_genes_s'].fillna(0, inplace=True)
output_df_GH['GH_genes_s'].replace('', 0, inplace=True)

print(output_df_GH[output_df_GH['GH_genes_ns'] != 0].head(2))


                                                                                 GH_genes_ns  \
libA_seq36960_[chr1:101150064-101150263]       VCAM1,lnc-VCAM1-1,piR-33432-028,piR-53029-003   
libA_seq255928_[chr1:101157210-101157409]  piR-59928-010,piR-53029-003,VCAM1,piR-33432-02...   

                                              GH_genes_s  
libA_seq36960_[chr1:101150064-101150263]   piR-33432-028  
libA_seq255928_[chr1:101157210-101157409]  piR-33432-028  


In [10]:
print(len(output_df_GH[output_df_GH['GH_genes_ns'] != 0]))

5442


In [11]:
# GH genes are a mix of ENSG and HGNC
# iterate through each gene and either convert or keep to new list
# first split into two columns

def split_ens_and_hgnc(r, col):
    genes = r[col]
    if not genes == 0:   
        genes_l = genes.split(',')
        ens_l = []
        hgnc_l = []
        for g in genes_l:
            if g.startswith('ENSG'):
                ens_l.append(g)
            else:
                hgnc_l.append(g)
        if len(ens_l) > 0:
            ens = ','.join(ens_l)
        if len(ens_l) == 0:
            ens = 0
        if len(hgnc_l) > 0:
            hgnc = ','.join(hgnc_l)
        if len(hgnc_l) == 0:
            hgnc = 0
        return ens, hgnc
    else:
        return 0, 0

output_df_GH[['GH_genes_ns_ens_split', 'GH_genes_ns_hgnc_split']] = \
    output_df_GH.apply(lambda row: pd.Series(split_ens_and_hgnc(row, 'GH_genes_ns')), axis=1)

output_df_GH[['GH_genes_s_ens_split', 'GH_genes_s_hgnc_split']] = \
    output_df_GH.apply(lambda row: pd.Series(split_ens_and_hgnc(row, 'GH_genes_s')), axis=1)

#print(output_df_GH[output_df_GH['GH_genes_ns'] == 0].head(5))

output_df_GH['GH_genes_ns_ens_split_conv'] = output_df_GH.apply(lambda row: convert(row, 'ens', 'GH_genes_ns_ens_split'), axis=1)
output_df_GH['GH_genes_ns_hgnc_split_conv'] = output_df_GH.apply(lambda row: convert(row, 'hgnc', 'GH_genes_ns_hgnc_split'), axis=1)
output_df_GH['GH_genes_s_ens_split_conv'] = output_df_GH.apply(lambda row: convert(row, 'ens', 'GH_genes_s_ens_split'), axis=1)
output_df_GH['GH_genes_s_hgnc_split_conv'] = output_df_GH.apply(lambda row: convert(row, 'hgnc', 'GH_genes_s_hgnc_split'), axis=1)

def merge_with_conv_col(r, col1, col2):
    g1 = r[col1]
    g2 = r[col2]
    if g1 != 0:
        g1_l = g1.split(',')
    if g1 == 0:
        g1_l = []
    if g2 != 0:
        g2_l = g2.split(',')
    if g2 == 0:
        g2_l = []
    g_l = g1_l + g2_l
    merged_s = ','.join(g_l)
    if len(merged_s) == 0:
        merged_s = 0
    return merged_s

output_df_GH['GH_genes_ns_HGNC'] = \
    output_df_GH.apply(lambda row: merge_with_conv_col(row, 'GH_genes_ns_hgnc_split', 'GH_genes_ns_ens_split_conv'), axis=1)

output_df_GH['GH_genes_s_HGNC'] = \
    output_df_GH.apply(lambda row: merge_with_conv_col(row, 'GH_genes_s_hgnc_split', 'GH_genes_s_ens_split_conv'), axis=1)

output_df_GH['GH_genes_ns_ENS'] = \
    output_df_GH.apply(lambda row: merge_with_conv_col(row, 'GH_genes_ns_ens_split', 'GH_genes_ns_hgnc_split_conv'), axis=1)

output_df_GH['GH_genes_s_ENS'] = \
    output_df_GH.apply(lambda row: merge_with_conv_col(row, 'GH_genes_s_ens_split', 'GH_genes_s_hgnc_split_conv'), axis=1)


print(output_df_GH[output_df_GH['GH_genes_ns_ENS'] != 0].head(5))

                                                                                 GH_genes_ns  \
libA_seq36960_[chr1:101150064-101150263]       VCAM1,lnc-VCAM1-1,piR-33432-028,piR-53029-003   
libA_seq255928_[chr1:101157210-101157409]  piR-59928-010,piR-53029-003,VCAM1,piR-33432-02...   
libA_seq20072_[chr1:101196756-101196955]           HSALNG0005548,ENSG00000228086,VCAM1,GPR88   
libB_seq201217_[chr1:104059849-104060048]  AMY2B,RF00100-006,RN7SKP285,LOC101928436,piR-3...   
libA_seq100098_[chr1:104059992-104060191]  AMY2B,RF00100-006,RN7SKP285,LOC101928436,piR-3...   

                                                   GH_genes_s  \
libA_seq36960_[chr1:101150064-101150263]        piR-33432-028   
libA_seq255928_[chr1:101157210-101157409]       piR-33432-028   
libA_seq20072_[chr1:101196756-101196955]                VCAM1   
libB_seq201217_[chr1:104059849-104060048]  LOC101928436,RNPC3   
libA_seq100098_[chr1:104059992-104060191]  LOC101928436,RNPC3   

                                

In [12]:
print(len(output_df_GH[output_df_GH['GH_genes_ns_ENS'] != 0]))

5017


In [13]:
# eQTL data

output_df_eqtl = pd.DataFrame(index=df_vars.index.tolist())

def clean_eqtls(r):
    eqtls = r['eQTL_dist_1000bp_v8_genes_unique']
    if eqtls != 0:
        eqtls_l = list(set(eqtls.split(',')))
        eqtls_s = ','.join(eqtls_l)
    if eqtls == 0:
        eqtls_s = 0
    return eqtls_s

output_df_eqtl['eQTL_genes_ENS'] = df_vars.apply(lambda row: clean_eqtls(row), axis=1)
output_df_eqtl['eQTL_genes_HGNC'] = output_df_eqtl.apply(lambda row: convert(row, 'ens', 'eQTL_genes_ENS'), axis=1)

print(output_df_eqtl[output_df_eqtl['eQTL_genes_ENS'] != 0].head(5))

                                                                          eQTL_genes_ENS  \
libB_seq57029_[chr1:1136106-1136305]   ENSG00000175756,ENSG00000225905,ENSG0000016257...   
libA_seq229534_[chr1:1197948-1198147]  ENSG00000225905,ENSG00000162572,ENSG0000016007...   
libB_seq264324_[chr1:1203069-1203268]  ENSG00000175756,ENSG00000225905,ENSG0000016257...   
libB_seq75291_[chr1:1500965-1501164]   ENSG00000235098,ENSG00000215915,ENSG0000020509...   
libA_seq35918_[chr1:1839919-1840118]   ENSG00000235098,ENSG00000231050,ENSG0000024833...   

                                                                         eQTL_genes_HGNC  
libB_seq57029_[chr1:1136106-1136305]   TTLL10,RNF223,ANKRD65,INTS11,ACAP3,TTLL10-AS1,...  
libA_seq229534_[chr1:1197948-1198147]  CPTP,ANKRD65,INTS11,MXRA8,PERM1,ACAP3,TTLL10-A...  
libB_seq264324_[chr1:1203069-1203268]  CPTP,ANKRD65,INTS11,MXRA8,ACAP3,MRPL20-DT,SCNN...  
libB_seq75291_[chr1:1500965-1501164]   TTLL10,VWA1,ATAD3C,ANKRD65,SSU72,TMEM240,MRP

In [14]:
print(len(output_df_eqtl[output_df_eqtl['eQTL_genes_ENS'] != 0]))

9503


In [15]:
# promoter data
# for both ns and s lists: take all overlapping TSSs

output_df_prom = pd.DataFrame(index=df_vars.index.tolist())

df_vars['Overlapping_TSSs_Ensembl'] = df_vars['Overlapping_TSSs_Ensembl'].fillna(0)

output_df_prom['prom_ENS'] = df_vars['Overlapping_TSSs_Ensembl']
output_df_prom['prom_HGNC'] = output_df_prom.apply(lambda row: convert(row, 'ens', 'prom_ENS'), axis=1)

print(output_df_prom[output_df_prom['prom_HGNC'] != 0].head(5))

                                              prom_ENS  prom_HGNC
libB_seq51614_[chr1:1847036-1847235]   ENSG00000169885     CALML6
libA_seq139784_[chr1:2355646-2355845]  ENSG00000149527      PLCH2
libB_seq221362_[chr1:6244322-6244521]  ENSG00000116254       CHD5
libB_seq249667_[chr1:8887208-8887407]  ENSG00000229716  RPL23AP19
libA_seq305430_[chr1:9002429-9002628]  ENSG00000131686        CA6


In [16]:
print(len(output_df_prom[output_df_prom['prom_ENS'] != 0]))

1466


In [17]:
# final df for reporting gene lists, ns = not-stringent, s = stringent
df2 = pd.DataFrame(index=df_vars.index.tolist())
df2['oligo'] = df2.index.tolist()

# promoter (same between ns and s)
df2['promoter_HGNC'] = output_df_prom['prom_HGNC']
df2['promoter_ENS'] = output_df_prom['prom_ENS']

# HiC HGNC + ENSid
df2['ns_HiC_HGNC'] = output_df_HiC['genes']
df2['s_ESC_HiC_HGNC'] = output_df_HiC['esc_genes']
df2['s_Hob_HiC_HGNC'] = output_df_HiC['msc_genes']
df2['s_NPC_HiC_HGNC'] = output_df_HiC['npc_genes']

df2['ns_HiC_ENS'] = output_df_HiC['genes_ens']
df2['s_ESC_HiC_ENS'] = output_df_HiC['esc_genes_ens']
df2['s_Hob_HiC_ENS'] = output_df_HiC['msc_genes_ens']
df2['s_NPC_HiC_ENS'] = output_df_HiC['npc_genes_ens']

# GH HGNC + ENSid (s is not cell-type specific)
df2['ns_GH_HGNC'] = output_df_GH['GH_genes_ns_HGNC']
df2['s_GH_HGNC'] = output_df_GH['GH_genes_s_HGNC']

df2['ns_GH_ENS'] = output_df_GH['GH_genes_ns_ENS']
df2['s_GH_ENS'] = output_df_GH['GH_genes_s_ENS']

# eQTL, same between ns and s
df2['eQTL_HGNC'] = output_df_eqtl['eQTL_genes_HGNC']
df2['eQTL_ENS'] = output_df_eqtl['eQTL_genes_ENS']

print(df2.head(5))

                                                                       oligo  \
libB_seq57029_[chr1:1136106-1136305]    libB_seq57029_[chr1:1136106-1136305]   
libA_seq229534_[chr1:1197948-1198147]  libA_seq229534_[chr1:1197948-1198147]   
libB_seq264324_[chr1:1203069-1203268]  libB_seq264324_[chr1:1203069-1203268]   
libB_seq75291_[chr1:1500965-1501164]    libB_seq75291_[chr1:1500965-1501164]   
libA_seq35918_[chr1:1839919-1840118]    libA_seq35918_[chr1:1839919-1840118]   

                                      promoter_HGNC promoter_ENS ns_HiC_HGNC  \
libB_seq57029_[chr1:1136106-1136305]              0            0           0   
libA_seq229534_[chr1:1197948-1198147]             0            0           0   
libB_seq264324_[chr1:1203069-1203268]             0            0           0   
libB_seq75291_[chr1:1500965-1501164]              0            0           0   
libA_seq35918_[chr1:1839919-1840118]              0            0           0   

                                      

In [18]:
def combine(r, columns):
    genes = []
    for c in columns:
        g = r[c]
        if g != 0:
            g_l = g.split(',')
            genes.extend(g_l)
        if g == 0:
            continue
    genes_unique = list(set(genes))
    if len(genes_unique) > 0:
        combined = ','.join(genes_unique)
    if len(genes_unique) == 0:
        combined = 0
    return combined

df2['ns_final_HGNC'] = df2.apply(lambda row: combine(row, ['promoter_HGNC', 'ns_HiC_HGNC', 'ns_GH_HGNC', 'eQTL_HGNC']), axis=1)
df2['s_final_ESC_HGNC'] = df2.apply(lambda row: combine(row, ['promoter_HGNC', 's_ESC_HiC_HGNC', 's_GH_HGNC', 'eQTL_HGNC']), axis=1)
df2['s_final_Hob_HGNC'] = df2.apply(lambda row: combine(row, ['promoter_HGNC', 's_Hob_HiC_HGNC', 's_GH_HGNC', 'eQTL_HGNC']), axis=1)
df2['s_final_NPC_HGNC'] = df2.apply(lambda row: combine(row, ['promoter_HGNC', 's_NPC_HiC_HGNC', 's_GH_HGNC', 'eQTL_HGNC']), axis=1)

df2['ns_final_ENS'] = df2.apply(lambda row: combine(row, ['promoter_ENS', 'ns_HiC_ENS', 'ns_GH_ENS', 'eQTL_ENS']), axis=1)
df2['s_final_ESC_ENS'] = df2.apply(lambda row: combine(row, ['promoter_ENS', 's_ESC_HiC_ENS', 's_GH_ENS', 'eQTL_ENS']), axis=1)
df2['s_final_Hob_ENS'] = df2.apply(lambda row: combine(row, ['promoter_ENS', 's_Hob_HiC_ENS', 's_GH_ENS', 'eQTL_ENS']), axis=1)
df2['s_final_NPC_ENS'] = df2.apply(lambda row: combine(row, ['promoter_ENS', 's_NPC_HiC_ENS', 's_GH_ENS', 'eQTL_ENS']), axis=1)

In [19]:
print(len(df2[df2['ns_final_ENS'] != 0]))
print(len(df2[df2['ns_final_HGNC'] != 0]))


11207
11073


In [30]:
x = []
def find_num_genes_linked(r):
    genes = r['ns_final_ENS']
    if genes == 0:
        return
    #print(genes)
    genes_l = genes.split(',')
    #print(genes_l)
    num_genes = len(genes_l)
    #print(num_genes)
    x.append(num_genes)
df2.apply(lambda row: find_num_genes_linked(row), axis=1)
print(np.median(x))
print(np.mean(x))
print(pd.Series(x).value_counts().to_dict())

4.0
6.431694476666369
{1: 2447, 2: 1696, 3: 1254, 4: 977, 5: 813, 6: 599, 7: 481, 8: 404, 9: 314, 10: 251, 11: 216, 12: 176, 13: 162, 15: 151, 14: 135, 18: 125, 17: 94, 16: 93, 19: 86, 20: 68, 22: 65, 21: 62, 24: 57, 25: 55, 23: 51, 26: 49, 30: 40, 27: 35, 28: 31, 35: 21, 33: 20, 32: 17, 39: 15, 31: 14, 34: 12, 36: 10, 37: 10, 44: 9, 29: 9, 40: 8, 56: 6, 45: 6, 46: 5, 41: 5, 43: 4, 48: 4, 42: 4, 53: 4, 38: 4, 47: 4, 49: 3, 64: 2, 67: 2, 57: 2, 73: 2, 58: 2, 59: 2, 55: 2, 52: 2, 62: 2, 66: 1, 74: 1, 60: 1, 68: 1, 61: 1, 54: 1, 70: 1, 71: 1}


In [31]:
# Fill in blank gene connections for ns list with closest TSS (even if not within promoter range)

df_vars['ensembl_gene_id_TSS'].fillna(0, inplace=True)
df_vars['hgnc_symbol_TSS'].fillna(0, inplace=True)
def fill_in_blanks(r, col, typ):
    genes = r[col]
    oligo = r['oligo']
    if genes != 0:
        return genes
    if genes == 0: # cases where no genes were connected
        if typ == 'hgnc':
            last_resort = df_vars.loc[oligo, 'hgnc_symbol_TSS']
        if typ == 'ens':
            last_resort = df_vars.loc[oligo, 'ensembl_gene_id_TSS']
        if last_resort == 0:
            return 0
        if last_resort != 0:
            return last_resort

df2['ns_final_HGNC'] = df2.apply(lambda row: fill_in_blanks(row, 'ns_final_HGNC', 'hgnc'), axis=1)
df2['ns_final_ENS'] = df2.apply(lambda row: fill_in_blanks(row, 'ns_final_ENS', 'ens'), axis=1)


In [32]:
print(len(df2[df2['ns_final_ENS'] != 0]))


14037


In [ ]:
print(df2.head(10))

In [33]:
df2.to_csv(f'{p_d}/final_lists.txt', sep='\t')

In [ ]:
print(df2.head())